In [91]:
function house(x)
    sigma = dot(x[2:end], x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq      # reflect to positive x axis
    else
        v[1] -= sq      # reflect to negative x axis
    end

    beta = 2.0 * v[1]^2 / (v[1]^2 + sigma)
    v1 = v[1]

    for i=1:length(v)
        v[i] /= v1
    end
    # display(v)
        
    return beta, transpose(v)   # get column vector v
end

function factorize_QR_house(A)
    m, n = size(A)
    A_out = copy(A)
    
    for j=1:n
        vTA = zeros(n)      # stores beta * vT * A
        beta, v = house(A[j:end, j]) 
        # v is of shape (m-j+1 x 1), same as the lower jth column of A

        Identity = Matrix(I, length(v), length(v))
        v_vec = zeros(length(v),1)
        
        for i=1:length(v)
            v_vec[i] = v[i]
        end
        
        QT = Identity - beta * v_vec * v_vec'  
        QT = pad(QT, m)
        
        display(QT)

        for k=j:n       # row indices of A, starts from diagonal element
            for i=j:m   
                vTA[k] += v[i-j+1] * A[i,k]
            end
            vTA[k] *= beta
        end

        for k=j:n
            for i=j:m
                A[i,k] -= v[i-j+1] * vTA[k]
            end
        end
        
        # display(j)
        # display(round.(A, digits=10))
        # display(v)
        A_out[j+1:end, j] = v[2:end]
    end

    return A_out
end

factorize_QR_house (generic function with 1 method)

In [100]:
function pad(QT, m)
    full = Matrix(I, m, m)
    current = size(QT)
    full[m-current:end,m-current:end] = QT
end

pad (generic function with 1 method)

In [101]:
factorize_QR_house(A)

MethodError: MethodError: no method matching -(::Int64, ::Tuple{Int64, Int64})

Closest candidates are:
  -(::Real, !Matched::Complex{Bool})
   @ Base complex.jl:321
  -(::Number, !Matched::UniformScaling)
   @ LinearAlgebra C:\Users\bhavy\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\uniformscaling.jl:147
  -(::Number, !Matched::Missing)
   @ Base missing.jl:124
  ...


In [51]:
function givens(a, b)
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a / b
            s = 1.0 / sqrt(1.0 + tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0 / sqrt(1.0 + tau*tau)
            s = c*tau
        end
    end
    return (c, s)
end

function factorize_QR_givens(A)
    m, n = size(A)
    A_out = copy(A)

    G = Matrix(I, n, n)

    for j=1:n
        c, s = givens(A[1,j], A[j+1,j])

        G[1,j] = c
        G[1,j+1] = -s
        G[j+1,j] = s 
        G[j+1,j+1] = c

        display(G * A)

        for k=j:n
            A_out[1,k], A_out[j+1,k] = c * A[1,k] - s * A[j+1,k], s * A[1,k] + c * A[j+1,k]
        end
        # display(j)
        # display(round.(A, digits=10))
    end
    return A_out
end

factorize_QR_givens (generic function with 1 method)

In [52]:
using LinearAlgebra
using Random

Random.seed!(69)

e = 1e-10 * randn()
A0 = [1 1 1; e 0 0; 0 e 0; 0 0 e]

4×3 Matrix{Float64}:
 1.0          1.0          1.0
 8.44369e-11  0.0          0.0
 0.0          8.44369e-11  0.0
 0.0          0.0          8.44369e-11

In [53]:
A = copy(A0)
A_house = factorize_QR_house(A)
A = copy(A0)
A_givens = factorize_QR_givens(A)
;

MethodError: MethodError: no method matching -(::Matrix{Bool}, ::Float64)
For element-wise subtraction, use broadcasting with dot syntax: array .- scalar

Closest candidates are:
  -(!Matched::Complex{Bool}, ::Real)
   @ Base complex.jl:326
  -(!Matched::Missing, ::Number)
   @ Base missing.jl:123
  -(!Matched::BigFloat, ::Union{Float16, Float32, Float64})
   @ Base mpfr.jl:506
  ...


In [46]:
display(A_house)
display(A_givens)

4×3 Matrix{Float64}:
 1.0           1.0        1.0
 4.22185e-11   0.0        0.0
 0.0          -0.414214   0.0
 0.0          -0.0       -0.517638

4×3 Matrix{Float64}:
 1.0   1.0           1.0
 0.0  -8.44369e-11  -8.44369e-11
 0.0   0.0          -8.44369e-11
 0.0   0.0           0.0